In [1]:
import requests
import os 
import json
import pandas as pd
import matplotlib.pyplot as plt
import time
import sqlite3

In [2]:
# Load apikey data

os.chdir('/Users/joemccann/MLProjects/alpaca_practice')
api_keys = json.load(open('apikey.json', 'r'))

In [3]:
# creates creates a connection to the database; will create a database if there is no connection
db_name = 'historical_data'
db_loc = '/Users/joemccann/MLProjects/alpaca_db/{}.db'.format(db_name)
db = sqlite3.connect(db_loc)

In [11]:
# Database functions
def create_table(symbol, db_columns, df):
    c = db.cursor()
    
    table_query = "CREATE TABLE IF NOT EXISTS {} ({});".format(symbol, db_columns)

    c.execute(table_query)

    df.to_sql(symbol, db, index=False, if_exists='replace')
        
    try:
        db.commit()
    except:
        db.rollback()


# Historical data functions
def get_bar_data(api_keys, url, symbols, start='', end='', limit=1000, timeframe='1Day'):
    list_of_dfs = []
    
    for symbol in symbols:
        headers = {
            "accept": "application/json",
            "APCA-API-KEY-ID": api_keys['APCA-API-KEY-ID'],
            "APCA-API-SECRET-KEY":  api_keys['APCA-API-SECRET-KEY']
        }
        
        params = {'symbols': symbol, 
              'timeframe':timeframe, 
              'start':start, 
              'end': end,
              'limit': limit}
    
        response = requests.get(url, headers=headers, params=params)
    
        data = response.json()
    
        df = pd.DataFrame(data['bars'][symbol])
        df.columns = ['close', 'high', 'low', 'trade_count', 'open', 'time', 'volume', 'volume_weighted_avg']
        df['time'] = pd.to_datetime(df['time'])
        df['stock_name'] = symbol
        df = df[['stock_name'] + [col for col in df.columns if col != 'stock_name']]
        df = df[[ 'time', 'stock_name','open', 'close', 'high', 'low', 'trade_count', 'volume', 'volume_weighted_avg']]

        list_of_dfs.append(df)

        # Create table if does not exist and add dataframe to it
        db_columns = "timestamp time primary key, stock_name TEXT, open REAL, close REAL, high REAL, low REAL, trade_count INTEGER, volume INTEGER, volume_weighted_avg REAL"
        create_table(symbol, db_columns, df)

    combined_df = pd.concat(list_of_dfs, ignore_index=True)

    

    return combined_df

def get_historical_quotes(api_keys, url, symbols, start='', end='', limit=1000):

    list_of_dfs = []

    for symbol in symbols:
        headers = {
            "accept": "application/json",
            "APCA-API-KEY-ID": api_keys['APCA-API-KEY-ID'],
            "APCA-API-SECRET-KEY":  api_keys['APCA-API-SECRET-KEY']
        }
        
        params = {'symbols': symbol, 
              'start':start, 
              'end': end,
              'limit': limit}
    
        response = requests.get(url, headers=headers, params=params)
    
        data = response.json()
    
        df = pd.DataFrame(data['quotes'][symbol])
        df.columns = ['ask_price', 'ask_size', 'ask_exchange', 'bid_price', 'bid_size', 'bid_exchange', 'condition_flag', 'timestamp', 'tape']
        df['stock_name'] = symbol
        df = df[['stock_name'] + [col for col in df.columns if col != 'stock_name']]
        df = df[['timestamp','stock_name', 'ask_price', 'ask_size', 'ask_exchange', 'bid_price', 'bid_size', 'bid_exchange', 'condition_flag', 'tape']]

        list_of_dfs.append(df)

    combined_df = pd.concat(list_of_dfs, ignore_index=True)

    return combined_df

# Latest data functions
def get_lastest_bar(api_keys, url, symbols):
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_keys['APCA-API-KEY-ID'],
        "APCA-API-SECRET-KEY":  api_keys['APCA-API-SECRET-KEY']
    }

    list_of_dfs = []

    for symbol in symbols:
        params = {'symbols': symbol}
        
        response = requests.get(url, headers=headers, params=params)
        
        data = response.json()
    
        df = pd.DataFrame(data['bars']).T
        df.columns = ['close', 'high', 'low', 'trade_count', 'open', 'timestamp', 'volume', 'volume_weighted_avg']
        df['stock_name'] = symbol
        df = df[['stock_name'] + [col for col in df.columns if col != 'stock_name']]
        df = df[['timestamp','stock_name', 'open', 'close', 'high', 'low', 'trade_count', 'volume', 'volume_weighted_avg']]
        
        list_of_dfs.append(df)
    
    combined_df = pd.concat(list_of_dfs, ignore_index=True)

    return combined_df

# Get Bar Data

In [13]:
url = "https://data.alpaca.markets/v2/stocks/bars?"
symbols = ['AMZN', 'TSLA', 'AAPL']
start = '2020-01-10'
end =  '2024-01-10'

get_bar_data(api_keys, url, symbols, start, end)

,time,stock_name,open,close,high,low,trade_count,volume,volume_weighted_avg
0,2020-01-10 05:00:00+00:00,AMZN,1905.37,1883.16,1906.94,1880.000,81924,3024534,1890.843009
1,2020-01-13 05:00:00+00:00,AMZN,1891.31,1891.30,1898.00,1880.800,78773,2992622,1890.957170
2,2020-01-14 05:00:00+00:00,AMZN,1885.88,1869.44,1887.11,1858.550,97323,3691756,1870.300572
3,2020-01-15 05:00:00+00:00,AMZN,1872.25,1862.02,1878.86,1855.090,80358,3056266,1864.995682
4,2020-01-16 05:00:00+00:00,AMZN,1882.99,1877.94,1885.59,1866.020,71727,2837080,1874.988524
...,...,...,...,...,...,...,...,...,...
2995,2023-12-22 05:00:00+00:00,AAPL,195.18,193.60,195.41,192.970,499840,37128763,194.101562
2996,2023-12-26 05:00:00+00:00,AAPL,193.61,193.05,193.89,192.830,488340,28921648,193.171311
2997,2023-12-27 05:00:00+00:00,AAPL,192.49,193.15,193.50,191.090,548205,48092035,192.567912
2998,2023-12-28 05:00:00+00:00,AAPL,194.14,193.58,194.66,193.170,472490,34056639,193.922138


# Bar DB test

In [14]:
# shows all table names in db
c = db.cursor()
c.execute('SELECT name FROM sqlite_master WHERE type="table";')
c.fetchall()

[('AMZN',), ('TSLA',), ('AAPL',)]

In [32]:
# Read data from database

table = 'AAPL'
query = "SELECT * FROM {};".format(table)

data = pd.read_sql_query(query, db)

In [33]:
data.head()

,time,stock_name,open,close,high,low,trade_count,volume,volume_weighted_avg
0,2020-01-10 05:00:00+00:00,AAPL,310.60,310.33,312.67,308.25,294491,36648233,310.681829
1,2020-01-13 05:00:00+00:00,AAPL,311.64,316.96,317.07,311.15,254955,32220012,313.781355
2,2020-01-14 05:00:00+00:00,AAPL,316.70,312.68,317.57,312.17,342961,42942708,314.712316
3,2020-01-15 05:00:00+00:00,AAPL,311.85,311.34,315.50,309.55,264906,32139593,312.905094
4,2020-01-16 05:00:00+00:00,AAPL,313.59,315.24,315.70,312.09,207425,29140071,314.205912


# Historical Quotes

In [24]:
url = "https://data.alpaca.markets/v2/stocks/quotes?"
# symbols = 'TSLA'
symbols = ['AMZN', 'TSLA', 'AAPL']
start = '2020-01-10'
end =  '2024-01-10'

quotes = get_historical_quotes(api_keys, url, symbols, start, end)

In [25]:
quotes.head()

,timestamp,stock_name,ask_price,ask_size,ask_exchange,bid_price,bid_size,bid_exchange,condition_flag,tape
0,2020-01-10T00:00:49.393Z,AMZN,1901.5,4,K,1901.00,1,T,[?],C
1,2020-01-10T00:02:56.395Z,AMZN,1901.5,4,K,1901.00,1,P,[?],C
2,2020-01-10T00:03:19.479Z,AMZN,1901.5,4,K,1901.00,1,T,[?],C
3,2020-01-10T00:04:47.359Z,AMZN,1901.5,3,K,1901.00,1,T,[?],C
4,2020-01-10T00:04:50.045Z,AMZN,1901.5,3,K,1901.04,1,K,[?],C


# Last Trade

In [26]:
last_url = "https://data.alpaca.markets/v2/stocks/bars/latest?"
symbols = ['AMZN', 'TSLA', 'AAPL']

latest_bar = get_lastest_bar(api_keys, last_url, symbols)

In [27]:
latest_bar.head()

,timestamp,stock_name,open,close,high,low,trade_count,volume,volume_weighted_avg
0,2024-02-02T20:59:00Z,AMZN,171.89,171.835,171.91,171.73,509,45593,171.836233
1,2024-02-02T20:59:00Z,TSLA,187.84,187.83,187.92,187.82,110,8009,187.862594
2,2024-02-02T21:01:00Z,AAPL,185.72,185.72,185.72,185.72,1,200,185.72


# Database practice

In [ ]:
# # creates creates a connection to the database; will create a database if there is no connection
# db = sqlite3.connect('/Users/joemccann/MLProjects/alpaca_db/historical_data.db')

# # In order to execute SQL statements and fetch results from SQL queries, we will need to use a database cursor. Call db.cursor() to create the Cursor
# c = db.cursor()

# table_name = 'amazon_historical_bars'
# columns = "timestamp time primary key, stock_name TEXT, open REAL, close REAL, high REAL, low REAL, trade_count INTEGER, volume INTEGER, volume_weighted_avg REAL"
# table_query = "CREATE TABLE IF NOT EXISTS {} ({});".format(table_name, columns)

# # execute allows you to run sql commands
# c.execute(table_query)

# # fetchall shows all table names
# c.execute('SELECT name FROM sqlite_master WHERE type="table";')
# c.fetchall()

# # Closes sqlite3; cannot delete a db file until this is run
# # c.close()